# Collaborative Multi-agent Learning with Deep Deterministic Policy Gradient (DDPG)

---

## Overview  

This agent implementation follows very closely the approach, network architectures, and hyperparameters found in the original DDPG paper mentioned below.  And being that this is a multi-agent environment, it also borrows the idea of a centralized critic network from the paper, _Mult-Agent Actor-Critic for Mixed Cooperative-Competitive Environments_. The centralized critic technique is one that has access to the full set of state observations and actions (observations and actions from both agents) of the environment vs. only the observation and action of a given agent.  With regard to the environment, we have 2 agents acting simultaneously, independantly, and cooperatively in the same environment.  As a practical matter, we really have only 1 instance of our MADDPG agent, and therefore only 1 set of networks (local and target for both actor and critic) to be updated and one common replay buffer of experiences being "shared" among the agents.  So each agent takes a distinct action from it's perspective in the environment and records it's experience into the replay buffer.  We also have access to the observations of both agents, however, at each time step, and so we also record this _full observation_ to the replay buffer to be used by the critic networks.  In addition, we add a random noise element to each of the agent's actions, which helps to broaden the exploration space. 

## Learning Algorithm

The Deep Deterministic Policy Gradient (DDPG) algorithm used in this repo is based on work first proposed in a paper titled, _Continuous Control With Deep Reinforcement Learning_. The architecture and hyperparameters found in that paper have guided the configuration of this agent's implementation.  In addition, this agent adds a minor expansion of the original DDPG agent by using a _centralized critic network_ to aid in training, which was proposed and detailed in the paper, _Mult-Agent Actor-Critic for Mixed Cooperative-Competitive Environments_.  Note that the actors always only have access to thier own observation, unlike the critic network, which has access to both agent's observations and actions.

A few key aspects of DDPG outlined in the paper are listed below: 

- DDPG is an approach that combines the strengths and stability techniques of DQN, such as use of a replay buffer and target networks, with the actor-critic method that allows DPG to handle continuous action spaces.   
  
  
- The actor network serves to map the state space to the specific (continuous) action to take, "skiping" the Q value approximation that happens with DQN.  The actor network is then updated using the policy gradient listed below.  
  
  
- The critic network then serves to "evaluate" the actor network's action by estimating the Q value of the state and action given by the actor network.  With a Q value, we can now measure the Bellman Loss (also given below), which tells our agent to "chase" or "avoid" the Q value one timestep in the future if it's higher or lower than the current Q value, respectively.
  
  
- Below is the loss function used to update the critic network.
![loss-function](images/critic_loss_func.PNG)  
  
  
- Below is the policy gradient used to update the actor network.
![loss-function](images/actor_loss_func.PNG)
  
  
- The DDPG algorithm pseudocode is given below.  
![algorithm](images/ddpg_algo.PNG)

- Multi-agent centralized critic, decentralized actor.  
![maddpg](images/maddpg_diagram.PNG)

---

## Network Achitecture

The architectures in this implementation is as follows:

**Actor**  
Input:     Linear(num_units = actor_observation_size)  
Hidden 1:  Linear(num_units = 256) > ReLU   
Hidden 2:  Linear(num_units = 128) > ReLU  
Output:    Linear(num_units = action size) > TanH  

**Critic**  
Input:     Linear(num_units = multi-actor_observation_size)  
Hidden 1:  Linear(num_units = 256) > relu  
Concat:    Concat(Critic-hidden1-output, Actor-output)  
Hidden 2:  Linear(num_units = 128 + action_size) > relu  
Output:    Linear(num_units = 1)  



## Hyperparameters

|Hyperparameter|Value|Description|
|--------------:|----:|:-----------|
|minibatch size| 256 | Number of training examples to sample from memory|
|replay buffer|200000|Number of experiences to store in memory|
|gamma|0.99|Discount factor of gamma used in Q-learning update|
|actor learning rate|1e-4|The learning rate used by Adam|
|critic learning rate|3e-4|The learning rate used by Adam|
|tau|1e-2|The parameters used by soft update of target network weights|
|L2 weight decay|0|weight decay used by Adam|
|max timesteps|2000|max number of timesteps for each episode|

## Plot of Rewards  

The plot below illustrates the agent's performance over time (or number of episodes). This environment is considered solved when the agent acheives an average score of at least 0.5 over 100 consecutive episodes. We can see that our **MADDPG agent was able to solve the environment in less than 1,100 episodes**

![Performance](images/maddpg_results.PNG)


***

## Ideas for Future Work

Additional thoughts for future work related to multi-agent environments include:  

1. **Competitive Environments**: Extend the MADDPG approach to a competitive scenario, like the Unity Soccer environment mentioned in the README.  
  
  
2. **Benchmarking**: This MADDPG agent is in fact very similar to a non-multi-agent DDPG.  It would be interesting to compare our performance (speed of learning and high scores) here with a plain DDPG, expanded multi-agent techniques, as well as other RL algorithms, like PPO
  
  
3. **Priortized Experience Replay**: rather than a uniform random selection of experiences to learn from, we would assign weighted probabilities to the experiences in order to prioritize experences with the greater losses, for example...  
  